In [6]:
### 주제: "산불 확산 패턴을 통한 피해 면적 예측 프로젝트"
# 핵심 키워드 정리
# 산불 확산 패턴 - 산불의 이동 및 확산 경로 분석
# 피해 면적 예측 - 산불로 인한 최종 피해 면적(km²)을 수치로 예측
# 위성 데이터 - NASA FIRMS(산불)와 POWER API(기상) 활용
# 머신러닝 모델	회귀 분석 기반 피해 예측 모델 개발

### 대략적 일정
## 1일차 (월요일)
# 오전 (4h): NASA FIRMS 데이터 수집 → 4명 분담: 위도/경도, brightness 등 확인 (pandas.read_csv)
#          : SUOMI VIIRS C2(VNP14IMG) + J1 VIIRS C2(VJ114IMG) 데이터 수집 → 위도/경도, FRP 확인.
# 오후 (4h): NASA POWER API 다운로드 → 기온, 풍속, 습도 체크 후 병합 시작 (pd.merge)
#          : NASA POWER API 기상 데이터 병합 → 기온, 풍속 등 체크.
# 화요일 대비 - GeoJSON 데이터 찾아보고 없으면 shapely(시각화에서 활용되는 라이브러리(사실 나도 잘 모름...))으로 다각형 생성 준비
# MODIS C6.1도 과거 참고용으로 다운로드 시작

## 2일차 (화요일)
# 오전 (4h): 데이터 전처리 → 결측치(평균값), 이상치 제거 (IQR)
# 오후 (4h): 탐색적 데이터 분석(EDA) + Folium → 상관관계 분석 (seaborn) 후 VIIRS C2로 산불 위치 맵 생성 (folium.Marker).
# Folium 사용 이유:
# 패턴 파악: 산불 분포를 지도로 시각화해 지역별 확산 경향 조기 확인
# 발표 준비: PPT용 기초 지도를 만들어 5일차 부담 줄임
# GPT 팁! - heatmap으로 brightness 강도 추가하면 패턴 더 선명해짐!

## 3일차 (수요일)
# 오전 (4h): 초기 모델 구축 → 선형 회귀, Ridge, Lasso 학습 (scikit-learn), RMSE/MAE 계산
# 오후 (4h): 피처 엔지니어링 → 풍속습도, 온도강수량 등 추가, 그래프 분석 (Folium 생략)
# 피처 엔지니어링 요약
#  - 데이터를 더 똑똑하게 가공해서 모델 성능 높이는 작업
#  - 풍속*습도, 고온 여부 등 산불 피해와 관련성 높은 변수 생성
#  - VIIRS의 FRP 활용해 피해 강도 변수 만들어보기

# 4일차 (목요일)
# 오전 (4h): 모델 개선 → Random Forest, XGBoost 등 추가 학습 및 튜닝 (GridSearchCV)
# 오후 (4h): 결과 분석 + Folium → VIIRS C2 최적 모델의 예측 피해 면적(km²)을 색상 원으로 맵에 추가 (folium.Circle)
# Folium 사용 이유: 최종 예측 결과를 시각화해 발표용 핵심 자료 완성, 모델별 성능 비교 가능
# 최고 모델의 예측만 빨간색으로 표시하면 시각적 임팩트 추가하기
# MODIS C6.1 예측과 비교 슬라이드 넣으면 깊이 추가

# 5일차 (금요일)
# 오전 (4h): PPT 완성 → Folium(VIIRS C2) 지도 2개(위치, 예측)와 모델 성능 표 삽입.
# 오후 (4h): 완성 되지 못한 부분 보완 및 PPT 발표 대본 및 연습


# PPT 구성
# 표지: "산불 피해 예측 프로젝트" – 팀명
# 소개: 산불 피해의 심각성 (통계 그래프).
# 데이터: NASA FIRMS + POWER API 개요, Folium 산불 위치 맵
# 방법: 전처리 → 다중 회귀 모델 → 검증 다이어그램
# 모델 비교: 선형 회귀, Random Forest 등 RMSE/MAE 표
# 결과: 예측 면적(km²) + Folium 피해 범위 맵 (4일차), 최적 모델 성능
# 결론: 기대 효과와 확장 가능성 (예: 실시간 예측)

###
# SUOMI VIIRS C2: 최신, 고해상도 산불 데이터의 주력
# J1 VIIRS C2   : SUOMI와 짝꿍으로 더 촘촘한 관측
# MODIS C6.1    : 과거를 돌아보는 데 유용한 백업

### SUOMI VIIRS C2 (Suomi NPP VIIRS Collection 2)
# SUOMI VIIRS는 Suomi National Polar-orbiting Partnership (Suomi NPP)라는 위성에 탑재된 VIIRS(Visible Infrared Imaging Radiometer Suite) 센서에서 나온 데이터
# 특징:
#  - 해상도     : 375m (산불 탐지에 더 세밀함).
#  - 관측 시간  : 하루에 오전 1:30쯤과 오후 1:30쯤, 지구를 두 번 커버
#  - 업데이트   : 최신 알고리즘으로 데이터를 재처리해서 더 정확함.
#  - 산불 데이터: NASA FIRMS에서 제공하는 "VNP14IMG"라는 이름으로 산불 위치와 강도(FRP: Fire Radiative Power)를 탐지
#  - 쉽게 말하면: Suomi NPP라는 위성이 찍은 최신 산불 사진첩

# J1 VIIRS C2 (NOAA-20 VIIRS Collection 2)
# J1 VIIRS는 NOAA-20 위성(원래 이름은 JPSS-1)에 탑재된 VIIRS 센서 데이터
# 특징:
#  - 해상도: SUOMI VIIRS와 동일한 375m.
#  - 관측 시간: 오전 2:20쯤과 오후 2:20쯤으로, SUOMI보다 50분 뒤에 지나감.
#  - 목적: SUOMI와 비슷하지만, 더 자주 관측해서 데이터 빈틈을 줄임.
#  - 산불 데이터: "VJ114IMG"라는 이름으로 FIRMS에서 제공. SUOMI와 거의 똑같은 정보를 주지만, 시간 차이 때문에 보완적인 역할.
#  - 쉽게 말하면: NOAA-20 위성이 SUOMI의 동생처럼 뒤따라가며 찍은 산불 사진첩

# MODIS C6.1 (MODIS Collection 6.1)
# MODIS는 Terra와 Aqua라는 두 위성에 탑재된 Moderate Resolution Imaging Spectroradiometer 센서의 데이터야. "C6.1"은 Collection 6.1, 즉 6번째 버전의 업데이트판.
# 특징:
# 해상도: 1km (VIIRS보다 덜 세밀함).
# 관측 시간: Terra는 오전 10:30쯤, Aqua는 오후 1:30쯤 지나감.
# 역사: 2000년대 초반부터 데이터를 쌓아왔으니 더 긴 시간 기록을 볼 수 있음.
# 산불 데이터: "MCD14"라는 이름으로 FIRMS에서 제공. 산불 위치와 FRP를 탐지하지만, 해상도가 낮아서 작은 불은 놓칠 수 있어.
# 쉽게 말하면: 오래된 형님 위성이 찍은 산불 사진첩의 6.1번째 버전이야. 좀 더 흐릿하지만 오래된 이야기를 들려줘!

# 차이점        비교
# 항목          SUOMI VIIRS C2      J1 VIIRS C2         MODIS C6.1
# 위성          Suomi NPP           NOAA-20 (JPSS-1)    Terra & Aqua
# 센서          VIIRS               VIIRS               MODIS
# 해상도        375m                375m                1km
# 관측 시간     1:30 AM/PM          2:20 AM/PM          10:30 AM, 1:30 PM
# 버전          Collection 2        Collection 2        Collection 6.1
# 장점          세밀하고 최신       SUOMI와 보완        긴 시간 기록
# 단점          짧은 역사           짧은 역사           덜 세밀함
# FIRMS 이름    VNP14IMG            VJ114IMG            MCD14

# SUOMI VIIRS C2와 J1 VIIRS C2: 최신이고 해상도가 높아서 산불 위치와 피해 면적 예측에 더 정확. 특히 375m 해상도는 작은 불도 잡아내니까 EDA와 Folium 시각화에 딱!
# MODIS C6.1: 과거 데이터(20년 이상)를 보고 싶을 때 유용해. 예를 들어, "과거 산불 패턴이 지금이랑 얼마나 달라졌나?" 같은 분석에 쓰면 좋음.
# 추천: 프로젝트에선 주로 VIIRS C2(SUOMI + J1)를 쓰고, 필요하면 MODIS C6.1로 과거 비교

In [7]:
### 1. 산불 데이터 (fire_data)
# 필수 데이터 (NASA FIRMS 활용)
# fire_id	    str	        산불 고유 ID
# latitude	    float	    위도
# longitude	    float	    경도
# date	        str	        관측 날짜 (YYYY-MM-DD)
# brightness	float	    화재 밝기 지수 (K)
# confidence	int	        신뢰도 (0~100)
# frp	        float	    화재 방출력 (MW)

### 2. 기상 데이터 (weather_data)
# 필수 데이터 (NASA POWER API 활용)
# latitude	    float	    위도
# longitude	    float	    경도
# date	        str	        기상 데이터 날짜 (YYYY-MM-DD)
# temperature	float	    기온 (°C)
# humidity	    float	    상대 습도 (%)
# wind_speed	float	    풍속 (m/s)
# precipitation	float	    강수량 (mm)

### 3. 산불 확산 경계 데이터 (geojson_data)
# region_name	str	        지역명 (ex: California)
# geometry	    dict	    GeoJSON 형식의 다각형 좌표
# fire_count	int	        해당 지역에서 발생한 산불 수

# 산불 경계 데이터 (GeoJSON)
# GeoJSON을 찾아봐야 할 곳
# Global Fire Atlas: https://www.globalfiredata.org
# NASA FIRMS (GIS Data): https://firms.modaps.eosdis.nasa.gov/gisdata/
# OpenStreetMap Wildfire Data: https://overpass-turbo.eu
# 없으면 산불데이터로 형성 (open ai 질문 결과 shapely 라이브러리 활용, 산불 데이터 (latitude, longitude)를 Convex Hull 알고리즘을 이용해 다각형(Polygon)으로 변환

### 4. 예측 피해 면적 데이터 (pred_data)
# latitude	    float	    예측된 피해 중심 위도
# longitude	    float	    예측된 피해 중심 경도
# pred_area	    float	    예측된 피해 면적 (km²)
# confidence	float	    예측 신뢰도 (0~1)

# 공통 사용 데이터 변수명
# fire_data: NASA FIRMS 데이터
# weather_data: NASA POWER API 기상 데이터
# geojson_data: 산불 확산 경계 데이터 (GeoJSON)
# pred_data: 머신러닝 모델의 예측 결과

### 필요할거로 예상되는 파일 구조
# wildfire_project/
# - data/
#   - fire_data.csv  # NASA FIRMS 데이터
#   - weather_data.csv  # NASA POWER API 데이터
#   - fire_boundary.geojson  # 산불 확산 경계 (GeoJSON) (없으면 위도 경도 직접 활용해서 사용)

# - models/
#   - 우리가 배운 회귀 모델들 사용 후 선정 할 파일
#     (클래스로 만들어서 데이터 프레임으로 하거나 모델별로 파일 생성.) - 상의 후 결정

# - visualizations/
#   - fire_map.html  # Folium 지도 (산불 위치)
#   - fire_heatmap.html  # Folium 히트맵 (산불 강도)

# - main.ipynb  # 분석 코드
# - presentation.pptx  # 최종 발표 자료

In [8]:
# SUOMI VIIRS C2 (VNP14IMG): 해상도 375m로 세밀한 관측이 가능하고 최신 알고리즘으로 처리된 데이터입니다.# J1 VIIRS C2 (VJ114IMG): 해상도 375m로 비슷하지만, SUOMI VIIRS C2와 시간 차이로 보완적인 역할을 합니다.
# 추천 방법
# 주요 분석이 실시간 산불 확산을 추적하고 예측하는 것이라면 실시간 데이터를 사용하세요.
# 과거 데이터를 사용할 경우, SUOMI VIIRS C2 (wildfire_aS1_df)와 J1 VIIRS C2 (wildfire_aJ1_df)를 비교하고 분석할 수 있습니다.
# 따라서 실시간 산불 확산 추적과 피해 면적 예측이 주 목표라면, wildfire_nS1_df (SUOMI VIIRS C2 실시간 데이터)와 wildfire_nJ1_df (J1 VIIRS C2 실시간 데이터)를 주로 사용할 수 있습니다.

# 데이터 병합에 대한 참고
# 병합 기준: 위도, 경도, 시간을 기준으로 데이터를 병합합니다. 이 두 데이터 세트를 병합하여, 실시간 산불 정보와 기상 데이터를 통합해 예측 모델을 구성하는 방식으로 진행할 수 있습니다.

# 예시 병합 코드:

# python
# 복사
# # 예시: 실시간 산불 데이터와 날씨 데이터 병합
# import pandas as pd

# # 산불 데이터 (실시간)
# wildfire_data = pd.concat([wildfire_nS1_df, wildfire_nJ1_df])

# # 기상 데이터 (POWER API에서 가져온 기온, 풍속, 습도 데이터)
# weather_data = pd.read_csv("power_api_data.csv")ㅁ

# # 날짜 및 위치를 기준으로 병합
# merged_data = pd.merge(wildfire_data, weather_data, on=["latitude", "longitude", "time"], how="inner")

# print(merged_data.head())
# 이렇게 실시간 산불 데이터와 기상 데이터를 병합하여, 이후 피해 면적 예측을 위한 모델링에 사용할 수 있습니다.

# 결론
# 실시간 산불 데이터인 wildfire_nS1_df와 wildfire_nJ1_df를 사용하되, 과거 데이터를 활용하여 패턴 분석을 원한다면 wildfire_aS1_df와 wildfire_aJ1_df를 비교하는 방식으로 진행할 수 있습니다.

# 기상 데이터와의 병합은 위도, 경도, 시간을 기준으로 진행하면 됩니다.

# 선택하신 데이터에 따라, 그에 맞는 분석을 진행하시면 됩니다! 추가적인 도움이 필요하면 언제든지 말씀해 주세요.



### 라이브러리 정의

In [2]:
import pandas as pd

### 시각화 라이브러리 정의
# - 파이썬에서 사용되는 기본 시각화 라이브러리
import matplotlib.pyplot as plt

# - 히트맵 라이브러리
import seaborn as sns
import geopandas as gpd
import folium
from shapely.geometry import Polygon

import os
from dask.diagnostics import ProgressBar
import dask.dataframe as dd

import numpy as np
from sklearn.cluster import KMeans
import requests
import time
import io

from scipy.interpolate import RBFInterpolator

from math import radians, sin, cos, sqrt, atan2


### 한글처리
plt.rc("font", family="Malgun Gothic")

# - 마이너스 기호 깨짐 처리
plt.rcParams["axes.unicode_minus"] = False

In [5]:
# 캘리포니아 경계 좌표
lat_range = (32.5, 42.0)
lon_range = (-124.5, -114.0)

# 0.1도 간격 격자 생성 (약 10km)
lat_points = np.arange(lat_range[0], lat_range[1], 0.1)
lon_points = np.arange(lon_range[0], lon_range[1], 0.1)
grid = [(lat, lon) for lat in lat_points for lon in lon_points]
grid_df = pd.DataFrame(grid, columns=["latitude", "longitude"])

# K-Means 클러스터링: 캘리포니아 좌표를 50개의 대표 좌표로 압축
# 목적: API 요청 횟수를 줄이고 데이터 크기를 줄이면서 지역적 대표성을 유지
kmeans = KMeans(n_clusters=50, random_state=42)
grid_df["cluster"] = kmeans.fit_predict(grid_df[["latitude", "longitude"]])
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=["latitude", "longitude"])

# 클러스터링 결과 확인
print("클러스터 중심 좌표 (50개 대표 좌표):")
print(cluster_centers.head())

# 클러스터 중심에서 기상 데이터 수집
start_date = "20150101"
end_date = "20250228"

def get_skiprows(response_text):
    lines = response_text.splitlines()
    skiprows = 0
    for line in lines:
        if line.strip() == "YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR":
            break
        skiprows += 1
    return skiprows

def get_weather_data(latitude, longitude):
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,WS2M,RH2M,PRECTOTCORR&community=AG&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=CSV"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        if response.status_code == 200:
            # 응답 데이터 구조 확인 (디버깅용)
            print(f"응답 데이터 상단 15줄 (위도: {latitude}, 경도: {longitude}):")
            print("\n".join(response.text.splitlines()[:15]))
            # 동적으로 skiprows 계산
            skiprows = get_skiprows(response.text)
            return pd.read_csv(io.StringIO(response.text), skiprows=skiprows, header=0)
        else:
            print(f"데이터 요청 실패! 상태 코드: {response.status_code}, 위도: {latitude}, 경도: {longitude}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"요청 오류 발생: {e}, 위도: {latitude}, 경도: {longitude}")
        return None

weather_data_list = []
for idx, row in cluster_centers.iterrows():
    print(f"요청 중: 클러스터 {idx} (위도 {row['latitude']}, 경도 {row['longitude']})")
    weather_data = get_weather_data(row["latitude"], row["longitude"])
    if weather_data is not None:
        weather_data["latitude"] = row["latitude"]
        weather_data["longitude"] = row["longitude"]
        weather_data_list.append(weather_data)
    time.sleep(5)  # API 요청 간 대기

# 데이터프레임 병합
weather_df = pd.concat(weather_data_list, ignore_index=True)

# acq_date 생성
weather_df["acq_date"] = pd.to_datetime(weather_df["YEAR"].astype(str) + weather_df["DOY"].astype(str), format="%Y%j")

# 결과 출력
print("\n기상 데이터 정보:")
print(weather_df.info())
print(weather_df.head(5))

# 저장
weather_df.to_csv("./USA data/clustered_weather_data.csv", index=False)

클러스터 중심 좌표 (50개 대표 좌표):
    latitude   longitude
0  35.900000 -118.729524
1  39.943750 -121.112981
2  37.559067 -114.773057
3  33.113174 -123.941317
4  37.182222 -123.942778
요청 중: 클러스터 0 (위도 35.90000000000005, 경도 -118.72952380952414)
응답 데이터 상단 15줄 (위도: 35.90000000000005, 경도: -118.72952380952414):
-BEGIN HEADER-
NASA/POWER Source Native Resolution Daily Data 
Dates (month/day/year): 01/01/2015 through 02/28/2025 in LST
Location: latitude  35.9   longitude -118.7295 
elevation from MERRA-2: Average for 0.5 x 0.625 degree lat/lon region = 1091.16 meters
The value for missing source data that cannot be computed or is outside of the sources availability range: -999 
parameter(s): 
T2M             MERRA-2 Temperature at 2 Meters (C) 
WS2M            MERRA-2 Wind Speed at 2 Meters (m/s) 
RH2M            MERRA-2 Relative Humidity at 2 Meters (%) 
PRECTOTCORR     MERRA-2 Precipitation Corrected (mm/day) 
-END HEADER-
YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR
2015,1,0.46,1.3,41.09,0.0
2015,2,3.28,1.05,

### 데이터 불러오기

In [4]:
# 1. 산불 데이터 로드
wildfire_dfs = {
    'aS1': pd.read_csv("./USA data/fire_archive_SV-C2_590694.csv"),
    'nS1': pd.read_csv("./USA data/fire_nrt_SV-C2_590694.csv"),
}
wildfire_dfs

{'aS1':         latitude  longitude  brightness  scan  track    acq_date  acq_time  \
 0       41.18757 -121.76961      303.88  0.44   0.38  2015-01-01       956   
 1       41.18763 -121.76556      297.14  0.44   0.38  2015-01-01       956   
 2       35.51954 -119.72784      302.88  0.39   0.36  2015-01-01       957   
 3       37.82349 -120.96186      299.62  0.39   0.37  2015-01-01       957   
 4       38.91062 -121.17077      303.10  0.40   0.37  2015-01-01       957   
 ...          ...        ...         ...   ...    ...         ...       ...   
 900591  32.63355 -114.53705      341.57  0.40   0.60  2024-12-31      1954   
 900592  32.63998 -114.53474      350.61  0.40   0.60  2024-12-31      1954   
 900593  35.57341 -115.48361      350.80  0.38   0.59  2024-12-31      1955   
 900594  35.53537 -115.45270      348.50  0.38   0.59  2024-12-31      1955   
 900595  35.34530 -120.36221      333.46  0.54   0.51  2024-12-31      2136   
 
        satellite instrument confidence  ve

In [6]:
# Haversine 공식으로 거리 계산
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# IDW(역거리 가중 보간) 함수
def idw_interpolation(row, weather_df, weather_vars):
    # 동일 날짜의 기상 데이터만 필터링
    weather_subset = weather_df[weather_df["acq_date"] == row["acq_date"]]
    if weather_subset.empty:
        return pd.Series([None] * len(weather_vars), index=weather_vars)
    
    distances = weather_subset.apply(
        lambda w: haversine(row["latitude"], row["longitude"], w["latitude"], w["longitude"]), axis=1
    )
    # 거리가 0인 경우를 방지하기 위해 최소값 설정
    distances = distances.replace(0, 1e-10)
    weights = 1 / (distances ** 2)
    weighted_sum = (weather_subset[weather_vars].multiply(weights, axis=0)).sum()
    return weighted_sum / weights.sum()

# 1. 산불 데이터 로드
wildfire_dfs = {
    'aS1': pd.read_csv("./USA data/fire_archive_SV-C2_590694.csv"),
    'nS1': pd.read_csv("./USA data/fire_nrt_SV-C2_590694.csv"),
}

# 2. 날씨 데이터 로드
weather_df = pd.read_csv("./USA data/clustered_weather_data.csv")

# 3. 성능 최적화: dtypes 최적화
# 메모리 사용량을 줄이기 위해 float32 사용
for df in wildfire_dfs.values():
    df["latitude"] = df["latitude"].astype("float32")
    df["longitude"] = df["longitude"].astype("float32")

weather_df["latitude"] = weather_df["latitude"].astype("float32")
weather_df["longitude"] = weather_df["longitude"].astype("float32")
weather_df["T2M"] = weather_df["T2M"].astype("float32")
weather_df["WS2M"] = weather_df["WS2M"].astype("float32")
weather_df["RH2M"] = weather_df["RH2M"].astype("float32")
weather_df["PRECTOTCORR"] = weather_df["PRECTOTCORR"].astype("float32")

# 4. 산불 데이터 처리
for df in wildfire_dfs.values():
    df["acq_date"] = pd.to_datetime(df["acq_date"])

wildfire_df = pd.concat(wildfire_dfs.values(), ignore_index=True)

# 5. 기상 데이터 처리
weather_df["acq_date"] = pd.to_datetime(weather_df["YEAR"].astype(str) + weather_df["DOY"].astype(str), format="%Y%j")

# 성능 최적화: 병합 전 필터링 (2025-03-01 이전 데이터만 사용)
cutoff_date = pd.Timestamp("2025-03-01")
weather_df = weather_df[weather_df["acq_date"] <= cutoff_date]
wildfire_df = wildfire_df[wildfire_df["acq_date"] <= cutoff_date]

# 6. 기상 데이터 보간 (클러스터별로 RBF 보간)
# 클러스터 식별을 위해 latitude와 longitude 조합으로 고유 키 생성
weather_df["cluster_id"] = weather_df["latitude"].astype(str) + "_" + weather_df["longitude"].astype(str)

# 보간 전 결측치 확인
print("보간 전 결측치:")
print(weather_df[["T2M", "WS2M", "RH2M", "PRECTOTCORR"]].isnull().sum())

# RBF 보간 적용
weather_vars = ["T2M", "WS2M", "RH2M", "PRECTOTCORR"]
for cluster_id in weather_df["cluster_id"].unique():
    cluster_data = weather_df[weather_df["cluster_id"] == cluster_id].copy()
    if cluster_data[weather_vars].isnull().any().any():
        # 시간 축 (acq_date를 숫자로 변환)
        time_points = (cluster_data["acq_date"] - cluster_data["acq_date"].min()).dt.days.values
        # 결측치가 없는 데이터만 사용
        for var in weather_vars:
            mask = ~cluster_data[var].isnull()
            if mask.sum() > 1:  # 최소 2개 이상의 데이터 포인트 필요
                rbf = RBFInterpolator(
                    time_points[mask].reshape(-1, 1),
                    cluster_data[var][mask],
                    kernel="thin_plate_spline"
                )
                # 결측치가 있는 위치에 대해 보간
                missing_mask = cluster_data[var].isnull()
                if missing_mask.any():
                    interpolated_values = rbf(time_points[missing_mask].reshape(-1, 1))
                    cluster_data.loc[missing_mask, var] = interpolated_values
        # 보간된 데이터로 업데이트
        weather_df.loc[weather_df["cluster_id"] == cluster_id, weather_vars] = cluster_data[weather_vars]

# 보간 후 결측치 확인
print("\nRBF 보간 후 결측치:")
print(weather_df[["T2M", "WS2M", "RH2M", "PRECTOTCORR"]].isnull().sum())

# 7. IDW 보간으로 기상 데이터 매핑
wildfire_df[weather_vars] = wildfire_df.apply(
    lambda row: idw_interpolation(row, weather_df, weather_vars), axis=1
)

# 병합된 데이터프레임 생성
merged_df = wildfire_df.copy()

# 병합 후 결측치 확인 (기상 데이터 매핑 검증)
print("\nIDW 적용 후 결측치:")
print(merged_df[weather_vars].isnull().sum())

# 8. 결측치 처리: 결측치가 있는 행은 선형 보간으로 채우기
for var in weather_vars:
    merged_df[var] = merged_df[var].interpolate(method='linear')

# 결측치 처리 후 확인
print("\n결측치 처리 후 결측치:")
print(merged_df[weather_vars].isnull().sum())

# 9. 2025년 3월 1일 이후 데이터 삭제 (필터링은 이미 적용됨, datetime 생성만 진행)
if "acq_time" in merged_df.columns:
    merged_df["datetime"] = pd.to_datetime(
        merged_df["acq_date"].astype(str) + " " + merged_df["acq_time"].astype(str).str.zfill(4),
        format="%Y-%m-%d %H%M"
    )
else:
    merged_df["datetime"] = merged_df["acq_date"]

# 10. 필요 없는 컬럼 삭제
columns_to_drop = ["acq_date", "acq_time"]
merged_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 11. 결과 출력
print("\n병합된 데이터 정보:")
print(merged_df.info())
print(merged_df.head(5))

# 12. 병합된 데이터 저장
merged_df.to_csv("./USA data/merged_wildfire_weather.csv", index=False)

보간 전 결측치:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64

RBF 보간 후 결측치:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64

IDW 적용 후 결측치:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64

결측치 처리 후 결측치:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64

병합된 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906078 entries, 0 to 906077
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   latitude     906078 non-null  float32       
 1   longitude    906078 non-null  float32       
 2   brightness   906078 non-null  float64       
 3   scan         906078 non-null  float64       
 4   track        906078 non-null  float64       
 5   satellite    906078 non-null  object        
 6   instrument   906078 non-null  object        
 7   confidence   906078 non-null  object      

In [7]:
# 병합된 데이터 정보 출력
print("\n병합된 데이터 정보:")
print(merged_df.info())

# 데이터 일부 출력 (병합이 제대로 되었는지 확인)
print("\n병합된 데이터 미리보기:")
print(merged_df.head(5))



병합된 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 906078 entries, 0 to 906077
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   latitude     906078 non-null  float32       
 1   longitude    906078 non-null  float32       
 2   brightness   906078 non-null  float64       
 3   scan         906078 non-null  float64       
 4   track        906078 non-null  float64       
 5   satellite    906078 non-null  object        
 6   instrument   906078 non-null  object        
 7   confidence   906078 non-null  object        
 8   version      906078 non-null  object        
 9   bright_t31   906078 non-null  float64       
 10  frp          906078 non-null  float64       
 11  daynight     906078 non-null  object        
 12  type         900596 non-null  float64       
 13  T2M          906078 non-null  float64       
 14  WS2M         906078 non-null  float64       
 15  RH2M         906078 n

In [8]:
# 결측치 확인
print("\n병합 후 결측치 확인:")
print(merged_df[weather_vars].isnull().sum())



병합 후 결측치 확인:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64
